In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%load_ext wurlitzer

The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer


In [14]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [15]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [16]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    config_path = "./",
                    data_path = "../data/",
                    offline = True,
                    environment  = "local",
                   sources = ["smart_citizen_kit"]
                    )

In [17]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

chains = [[s,] for s in sources]
for a in processors:
    if not isinstance(a.match, UUID): 
        chains.append([str(a.match), a,])
        continue
    for c in chains:
        matches = a.match.int == c[-1].id.int
        if matches: c.append(a)
            

for c in chains:
    print(" --> ".join(str(a) for a in c))

SmartCitizenKitSource --> CSVParser --> Splitter --> NewTimeAggregator --> ODCEncoder
[Match(state = 'odc_encoded')] --> RESTWriter()


In [18]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


id,33572aa3-4d8a-4d9e-b854-cdcc97015ef7
mappings,"[InputColumn(name='time', key='time', type=None, unit=None, discard=False, canonical_variable=Non..."
finish_after,None
copy_metadata_to_columns,"[station_name, station_id, lat, lon, created_at, city, country, author]"
cache_version,3
use_cache,True
cache_directory,/Users/math/git/IonBeam_bundle/data/inputs/smart_citizen_kit
name,value
source_action_id,33572aa3-4d8a-4d9e-b854-cdcc97015ef7
state,raw
mars_request,{}


Aggregators


id,207b2f3c-9efc-4a6c-9327-e1723d60d77f
match,5488f381-82d4-4be5-bd8e-72b2491c6502
granularity_hours,1
time_direction,forwards
min_emit_after_hours,96
emit_after_multiplier,5
time_chunks,{}
time_frontier,None
name,value
source_action_id,207b2f3c-9efc-4a6c-9327-e1723d60d77f
state,time_aggregated


Actions


id,ad234dd3-73c5-41fa-af9c-4359204bfbb5
match,33572aa3-4d8a-4d9e-b854-cdcc97015ef7
mappings,"[InputColumn(name='time', key='time', type=None, unit=None, discard=False, canonical_variable=Can..."
identifying_keys,[time]
metadata_keys,"[station_name, station_id, author, lat, lon, created_at, city, country]"
separator,","
custom_nans,None
name,value
source_action_id,ad234dd3-73c5-41fa-af9c-4359204bfbb5
state,parsed
mars_request,{}


id,5488f381-82d4-4be5-bd8e-72b2491c6502
match,ad234dd3-73c5-41fa-af9c-4359204bfbb5
identifying_keys,[time]
metadata_keys,"[station_name, author, lat, lon, created_at, city, country]"
name,value
source_action_id,5488f381-82d4-4be5-bd8e-72b2491c6502
state,parsed
mars_request,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,646fb44d-ffd0-4b73-b625-26b128a2693a
match,207b2f3c-9efc-4a6c-9327-e1723d60d77f
output,outputs/{source}/odb/{observation_variable}/{observation_variable}_{time_slice.start_time}.odb
MARS_keys,"[MARS_Key(name='class', dtype=<DataType.STRING: 3>, fill_method='constant', value='rd', key=None,..."
one_file_per_granule,True
columns_to_metadata,[]
seconds,True
minutes,True
name,value
source_action_id,646fb44d-ffd0-4b73-b625-26b128a2693a
state,odc_encoded


id,9291d3ff-453d-497d-ae4f-781696943e74
match,[Match(state = 'odc_encoded')]
name,value
source_action_id,9291d3ff-453d-497d-ae4f-781696943e74
state,written
mars_request,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
metkit_language_template,/Users/math/git/IonBeam_bundle/IonBeam/config/metkit/language.yaml.template
environment,local


In [19]:
online = True
if online:
    source = sources[0]
    source.globals.offline = !online
    source.use_cache = False
    incoming_message_stream = source.generate()
    message = next(incoming_message_stream)
    message
else:
    p = Path('/Users/math/git/IonBeam-Deployment/data/inputs/meteotracker/MeteoTracker_62ae154f1d8e11061d4474b2.csv')
    message = FileMessage(
        metadata=MetaData(state='raw',
            source='meteotracker',
            filepath= p,
            variables = list(pd.read_csv(p).columns),             
            ))
message

StopIteration: 

In [ ]:
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator

possible_actions = [a for a in actions if not isinstance(a, Source)]

message_history = []
while True:
    display(message)
    message_history.append(message)
    matching = [action for action in possible_actions if action.matches(message)]
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))
        

In [ ]:
m = message_history[-1]
m.data

In [ ]:

df = pd.DataFrame({"class" : ["rd", "rd"], "col1" : [1,2], "col2" : [1,2]})
pyodc.encode_odb(df, "test.odb")

with open("test.odb", "rb") as f:
    pyfdb.archive(f.read())